# Cosmic ray removal

Almost all images from a CCD will include some number of cosmic rays, charged particles which bombard the Earth's upper atmosphere. Some of those will make it through the atmosphere and into your detector (the rate of cosmic rays will be much higher for cameras in space). Although the number of cosmic rays is roughly proportional to exposure time, there will be cosmic rays even in bias frames in which the chip is immediately read out.

This notebook explains how to remove cosmic rays from calibration images and science images.

## Removal from calibration images

The easiest way to remove cosmic rays from calibration images (bias, dark and flat images) is to combine them properly. Cosmic rays are, by their nature, random events that will affect different parts of each of the calibration images. A pixel affected by a cosmic ray in one of the dark images, for example, will almost certainly *not* be affected by a cosmic ray in any of the other dark images. 

Combining those images by averaging (to reduce noise as much as possible) and sigma clipping (to exclude extreme pixels in individual images like the one with a cosmic ray) will eliminate the cosmic ray from the combined dark image. An alternative would be to combine the images using a median. A detailed description of each option is discussed in the [section on image combination](01.06-Image-combination.ipynb).

The method described below for removing cosmic rays from science images will not work well for removing them from calibration images and is unnecessary because they can be removed by properly combining the images.

## Removal from science images

One good technique for removing cosmic rays from an image is the [LACosmic method](http://www.astro.yale.edu/dokkum/lacosmic/) originally developed and implemented for IRAF by [Pieter G. van Dokkum](https://www.pietervandokkum.com/). The original paper describing the method, which uses the sharp edges of cosmic rays to distinguish them from other sources in the image, is [here](http://adsabs.harvard.edu/abs/2001PASP..113.1420V).


The specific implementation of LACosmic used here is the astropy affiliated package [Astro-SCRAPPY](https://github.com/astropy/astroscrappy). If you use this code to remove cosmic rays you should cite both the original paper and [Astro-SCRAPPY](https://github.com/astropy/astroscrappy) (citation details are on its web site). The code below never directly imports [Astro-SCRAPPY](https://github.com/astropy/astroscrappy) because ccdproc provides a wrapper for it so we called attention to it here.

### Very important notes about using LACosmic

There are a few things to be aware of before using the LACosmic technique. These are drawn from the advice van Dokkum provides under [Notes for Users](http://www.astro.yale.edu/dokkum/lacosmic/) and the original paper.

1. The images must be bias and dark subtracted.
2. The images should be flat fielded, though the technique can be applied without flat fielding.
3. The images should **not** have the sky subtracted before detecting the cosmic rays.
4. The noise level in the image needs to be accurately measured.
5. The image and the noise have to be in the same units, typically electrons.

In [ ]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt

from astropy.nddata import CCDData
from astropy import units as u
import ccdproc as ccdp
from photutils import detect_sources

from astrowidgets import ImageWidget

from convenience_functions import show_image

### Input image

The image we will use in this notebook is one of the reduced images from Example 2 in the previous notebooks. It is an image of the field of the exoplanet Kelt-16b, taken with a thermo-electrically cooled CCD with read noise of 10$e^-$ and gain of $1.5~e^-$/ADU.

In [ ]:
ex2_path = Path('example2-reduced')

ccd = CCDData.read(ex2_path / 'kelt-16-b-S001-R001-C084-r.fit')

In [ ]:
show_image(ccd, cmap='gray')

lights = ifc.summary['imagetyp'] == 'LIGHT'

ifc.summary[lights]

The unit of this image is ADU, so we need to multiply by the gain to convert to electrons.

In [ ]:
ccd = ccdp.gain_correct(ccd, 1.5 * u.electron / u.adu)

### Running LACosmic

The actual invocation of LACosmic is fairly straightforward. The key parameters are `readnoise`, the read noise, and `sigclip`, which determines how far above the background a pixel needs to be to consider it a cosmic ray. There is no hard-and-fast rule for selecting the proper value of `sigclip`. In the original paper a value of 5 is recommended, but for this image it finds 8,000 pixels contaminated by cosmic rays. That is not plausible for an image taken with a camera a thousand feet above sea level.

Higher values of `sigclip` reduce the number of cosmic rays found. The value used below, 8, seemed to work well for this image, finding a total of roughly 400 pixels that are cosmic rays, and a couple dozen candidate cosmic rays that extend across multiple pixels.

The function [`cosmicray_lacosmic`]() from ccdproc returns a new image in which the mask is `True` for pixels in which a cosmic ray was detected and `False` otherwise. The data in the new image has values in the pixels in which cosmic rays were identified replaced by interpolating the neighboring pixels. 

We will take a look at the cosmic rays identified by LACosmic in a moment.

In [ ]:
%%time
new_ccd = ccdp.cosmicray_lacosmic(ccd, readnoise=10, sigclip=8, verbose=True)

In [ ]:
new_ccd.mask.sum()

### Examining the cosmic rays identified by LACosmic

There are 428 pixels that have been flagged as cosmic rays. Looking through each of them individually would be tedious, at best. It would also presumably be difficult to decide visually if a single pixel tagged as a cosmic ray was actually a cosmic ray, but it would be helpful to look at the larger cosmic rays, i.e. those that span multiple pixels.

To identify those larger cosmic rays we will use the function `detect_sources` from the package [photutils](https://photutils.readthedocs.io), which identifies contiguous pixels in an image via image segmentation. Though [`detect_sources`]() is intended for detecting extended or stellar  sources in an image it happens to work very well for identifying extended cosmic rays in the mask generated by [`cosmicray_lacosmic`]().

The threshold below should be something less than 1 to ensure that only the masked pixels, i.e. those whose values are 1, are included as sources. The number of pixels is the number which must be adjacent (either by edge or by corner) to be considered a source.

In [ ]:
threshold = 0.5
n_pixels = 3
crs = detect_sources(new_ccd.mask, threshold, n_pixels)

We first check to see how many of the pixels identified by LACosmic are part of these extended cosmic rays.

In [ ]:
sum(crs.areas)

Looks like about 75% of them.

In [ ]:
ccd_dark = CCDData.read(ex2_path / 'combined_dark_90.000.fit')

In [ ]:
def image_snippet(image, center, width=50, axis=None, fig=None):
    x_slice = slice(center[1] - width // 2, center[1] + width // 2)
    y_slice = slice(center[0] - width // 2, center[0] + width // 2)
    #print('In snippet', x_slice, y_slice)
    sub_image = image[x_slice, y_slice]
    show_image(sub_image, cmap='gray', ax=axis, fig=fig, show_colorbar=True, figsize=(2, 2))
    

In [ ]:
def mid(sl):
    return (sl.start + sl.stop) // 2

n_rows = len(crs.slices)

fig, axes = plt.subplots(n_rows, 3, sharex=True, sharey='row', figsize=(40, 160), tight_layout=True)

for row, s in enumerate(crs.slices):
#for s in crs_no_sub.slices[-3:]:
    x = mid(s[1])
    y = mid(s[0])
    #print(x, y)
    for column, image in enumerate([new_ccd.mask, ccd, ccd_dark]):
        image_snippet(image, (x, y), width=80, axis=axes[row, column], fig=fig)

In [ ]:
iw = ImageWidget()
iw

In [ ]:
iw.load_array(new_ccd.mask)

In [ ]:
iw.center_on((3961, 2679))

#iw.center_on((3151, 481))

#iw.center_on((3054, 904))
#iw.center_on((3133.15, 1366.32))
iw.zoom_level = 4

In [ ]:
iw.zoom_level

In [ ]:
crs.slices

In [ ]:
crs_no_sub = detect_sources(new_ccd_no_sub.mask, 0.5, 3)

In [ ]:
crs_no_sub.slices

In [ ]:
def mid(sl):
    return (sl.start + sl.stop) / 2
from time import sleep
iw.zoom_level = 8
for s in crs.slices[-3:]:
#for s in crs_no_sub.slices[-3:]:
    x = mid(s[1])
    y = mid(s[0])
    print(x, y)
    iw.center_on((x, y))
    sleep(2)

In [ ]:
iw

In [ ]:
iw.load_array(ccd.data)

In [ ]:
ccd_raw = CCDData.read('example-thermo-electric/kelt-16-b-S001-R001-C084-r.fit.gz')

In [ ]:
crs.areas.sum()

In [ ]:
new_ccd.write(ex2_path / 'kelt-16-with-cr-mask.fits')